# import 및 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
distribution_centers = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/distribution_centers_optimized.parquet')
# events_file = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized.parquet')
inventory_items = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/inventory_items_optimized.parquet')
orders = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/orders_optimized.parquet')
order_items = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/order_items_optimized.parquet')
products = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/products_optimized.parquet')
users = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/users_optimized.parquet')

In [3]:
events1 = pd.read_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided1.parquet")
events2 = pd.read_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided2.parquet")
events = pd.concat([events1, events2]).sort_index()

# 집계기준에 따른 status비중
country가 기본

## 각각 집계해서 하나로 합치기

In [4]:
#각 집계기준별로 status 비중이 어떻게 되는지 파악
agg_columns = ['id', 'country'] # 집계 기준 부분
user_temp = users.sort_values(by = 'id')[agg_columns] #user에서 필요한 칼럼만 가져오기
#whole인 user의 주소 정보 확인 테이블 만들기
Whole_users_by_agg = (
  order_items
  .groupby('user_id')['order_id'].nunique().reset_index()
  .rename(columns = {'order_id' : 'n_of_Whole'})
  .merge(user_temp, left_on='user_id', right_on='id', how = 'left')
  .groupby(['country'])['n_of_Whole'].sum().reset_index()
  )

status_agg_merge_df = Whole_users_by_agg # 전체 합칠 df

for status_value in order_items['status'].unique():
  #processing 이면 shipped_at도 delivered_at도 returned_at도 없음
  status_temp_df = (
    order_items
    .query(f'status == "{status_value}"')
    .groupby('user_id')['order_id'].nunique().reset_index()
    .rename(columns = {'order_id' : f'n_of_{status_value}'})
    .merge(user_temp, left_on='user_id', right_on='id', how = 'left')
    .groupby(['country'])[f'n_of_{status_value}'].sum().reset_index()
    
    )
  status_agg_merge_df = status_agg_merge_df.merge(status_temp_df, on = 'country')

status_agg_merge_df

,country,n_of_Whole,n_of_Cancelled,n_of_Complete,n_of_Shipped,n_of_Processing,n_of_Returned
0,Australia,2630,382,663,762,551,272
1,Austria,2,0,1,1,0,0
2,Belgium,1441,187,349,472,287,146
3,Brasil,18262,2784,4612,5360,3675,1831
4,China,42986,6520,10713,12905,8535,4313
5,Colombia,19,3,6,4,4,2
6,Deutschland,1,0,1,0,0,0
7,España,4,0,1,0,2,1
8,France,5968,891,1460,1841,1162,614
9,Germany,5286,781,1319,1601,1098,487


## 전체로 나눠줘서 비중 구하기

In [6]:
#전체 대비 비중 구하기  # 아직 더 좋은 방법이 있을 것 같기도 한데 안 떠올라서 일단 여기서 hold
#문자열인 country 칼럼 일단 제외
country_column = status_agg_merge_df['country']
#country칼럼 제외한 나머지 칼럼들을 whole로 나눠주고 100곱해서 비중 구하기. 마지막에 소수 둘째자리에서 반올림
status_agg_merge_df = (status_agg_merge_df.iloc[:, 1:].div(status_agg_merge_df['n_of_Whole'], axis= 0)*100).round(2)
#insert함수로 첫번째 칼럼(0)으로 삽입. 이름도 다시 country
status_agg_merge_df.insert(0, 'country', country_column)
status_agg_merge_df

,country,n_of_Whole,n_of_Cancelled,n_of_Complete,n_of_Shipped,n_of_Processing,n_of_Returned
0,Australia,100.0,14.52,25.21,28.97,20.95,10.34
1,Austria,100.0,0.00,50.00,50.00,0.00,0.00
2,Belgium,100.0,12.98,24.22,32.76,19.92,10.13
3,Brasil,100.0,15.24,25.25,29.35,20.12,10.03
4,China,100.0,15.17,24.92,30.02,19.86,10.03
5,Colombia,100.0,15.79,31.58,21.05,21.05,10.53
6,Deutschland,100.0,0.00,100.00,0.00,0.00,0.00
7,España,100.0,0.00,25.00,0.00,50.00,25.00
8,France,100.0,14.93,24.46,30.85,19.47,10.29
9,Germany,100.0,14.77,24.95,30.29,20.77,9.21


In [7]:
'Deutschland' in users['country'].unique() # 값이 0이 나오는 것 같아서

True